In [2]:
import torch
import numpy as np
import pickle

In [5]:
#load data
model = "ResNet"
In_dataset="ImageNet"

prefix="stat/"+model+"/"
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
with open(prefix+In_dataset+"_"+"Intrain"+"_"+"features"+".pkl", 'rb') as f:
    train_data = pickle.load(f)
    for i in range(len(train_data)):
        train_data[i]=train_data[i].to(device)

with open(prefix+In_dataset+"_"+"Intrain"+"_"+"logits"+".pkl", 'rb') as f:
    train_result = pickle.load(f)
    for i in range(len(train_result)):
        train_result[i]=train_result[i].to(device)

with open(prefix+In_dataset+"_"+"Intest"+"_"+"features"+".pkl", 'rb') as f:
    test_data = pickle.load(f)
    for i in range(len(test_data)):
        test_data[i]=test_data[i].to(device)

with open(prefix+In_dataset+"_"+"Intest"+"_"+"logits"+".pkl", 'rb') as f:
    test_result = pickle.load(f)
    for i in range(len(test_result)):
        test_result[i]=test_result[i].to(device)



/home/zwling/.conda/envs/lzw/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [6]:
Ood_dataset="iNat"

with open(prefix+Ood_dataset+"_"+"Oodtest"+"_"+"features"+".pkl", 'rb') as f:
    ood_data = pickle.load(f)
    for i in range(len(ood_data)):
        #如果ood_data[i]是list，那么就转换成空tensor
        if len(ood_data[i])==0:
            ood_data[i]=torch.tensor([]).to(device)
        else:
            ood_data[i]=ood_data[i].to(device)


with open(prefix+Ood_dataset+"_"+"Oodtest"+"_"+"logits"+".pkl", 'rb') as f:
    ood_result = pickle.load(f)
    for i in range(len(ood_result)):
        if len(ood_result[i])==0:
            ood_result[i]=torch.tensor([]).to(device)
        else:
            ood_result[i]=ood_result[i].to(device)

In [14]:
import scipy.stats as stats

eps = 1e-5

train_mu = []
train_sigma = []
train_mean = []
print(len(train_data))
for i in range(len(train_data)):
    train_mean.append(torch.mean(train_data[i], dim=0))
    train_mu.append(train_data[i].mean())
    train_sigma.append(train_data[i].std())

print(train_mu)
print(train_sigma)

1000
[tensor(0.4282), tensor(0.4756), tensor(0.4603), tensor(0.4285), tensor(0.4187), tensor(0.4552), tensor(0.4715), tensor(0.4793), tensor(0.4994), tensor(0.4957), tensor(0.4242), tensor(0.4428), tensor(0.4666), tensor(0.4415), tensor(0.4237), tensor(0.4393), tensor(0.4884), tensor(0.4722), tensor(0.4593), tensor(0.4624), tensor(0.4548), tensor(0.4530), tensor(0.4693), tensor(0.4880), tensor(0.4734), tensor(0.3893), tensor(0.4258), tensor(0.3964), tensor(0.4110), tensor(0.4856), tensor(0.4503), tensor(0.4458), tensor(0.4380), tensor(0.4585), tensor(0.4653), tensor(0.4471), tensor(0.4600), tensor(0.4223), tensor(0.4196), tensor(0.4723), tensor(0.4238), tensor(0.3907), tensor(0.4430), tensor(0.5016), tensor(0.4318), tensor(0.4089), tensor(0.4198), tensor(0.4802), tensor(0.4686), tensor(0.4445), tensor(0.4609), tensor(0.5275), tensor(0.4132), tensor(0.4274), tensor(0.4085), tensor(0.4186), tensor(0.3925), tensor(0.3840), tensor(0.4232), tensor(0.4020), tensor(0.4037), tensor(0.4597), te

In [9]:
def ash_s(x, percentile=65):
    x = x.clone()
    b, c = x.shape

    # calculate the sum of the input per sample
    s1 = x.sum(dim=[1])
    n = x.shape[1:].numel()
    k = n - int(np.round(n * percentile / 100.0))
    t = x.view((b, -1 ))
    v, i = torch.topk(t, k, dim=1)
    t.zero_().scatter_(dim=1, index=i, src=v)

    # calculate new sum of the input per sample after pruning
    s2 = x.sum(dim=[1])

    # apply sharpening
    scale =  s1/s2

    return scale
from torch import nn
cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)


In [10]:

import numpy as np
from scipy.stats import norm
from scipy.stats import skew, kurtosis

def get_auc(k=0.1):
   
    k = int(train_data[0].shape[1] *k)
    labels = []
    scores = []
    for i in range(0,len(train_data)):
        mu = train_mu[i]
        sigma = train_sigma[i]
        in_data = test_data[i].to(device)
        
        od_data = ood_data[i].to(device)
        
         
        cv = sigma/mu
        error_rate = (train_mean[i] - in_data).norm(dim=1,p=1)/in_data.norm(dim=1,p=1)
        scale = ash_s(in_data, percentile=90)

            
        
        v,idx = torch.topk(in_data, k, dim=1,largest=False)
       

        output_logits = test_result[i].to(device) * torch.exp(scale.unsqueeze(1))

        output_logits = 1 * torch.logsumexp(output_logits / 1, dim=1).cpu().numpy()
        ind_scores =  -(cv*error_rate)
        if len(ood_data[i])>0:
            scale = ash_s(od_data, percentile=90)
           
            error_rate = (train_mean[i] - od_data).norm(dim=1,p=1)/od_data.norm(dim=1,p=1)




            output_logits = ood_result[i].to(device)* torch.exp(scale.unsqueeze(1))
            output_logits = 1 * torch.logsumexp(output_logits / 1, dim=1).cpu().numpy()

            ood_scores = -(cv*error_rate)
          
            ood_labels = np.zeros(ood_scores.shape)
            scores.append(ood_scores)
            labels.append(ood_labels)
           

        
        ind_labels = np.ones(ind_scores.shape)
        scores.append(ind_scores)
        labels.append(ind_labels)

    labels = np.concatenate(labels)
    scores = np.concatenate(scores)
    from utils.metrics import cal_metric
    print(labels.shape)
    print(scores.shape)
    auroc, aupr, fpr = cal_metric(labels, scores)
    return auroc, aupr, fpr
get_auc()

(60000,)
(60000,)


(0.975568514, 0.9948296864308895, 0.13)